## Employee_Number
- 1080 valores únicos (incluyendo el 0)
- 534 duplicados
- (1080 + 534 = 1614 totales)
- ...................
- dtype  -> object

## Duplicados

- 431 son 'Employee_Number'= 0
- 104 en las siguientes situaciones

- df_duplicados = 639 rows -> 431 'Employee_Number'=0 + (104 x 2)


- Situación 1: 51 employees
    - Todas las columnas son iguales
   
    ➡️ Solución: eliminar el primer registro   

- Situación 2: 13 employees
    - Distance_From_Home diferentes
   
    ➡️ Solución: eliminar el primer registro   

- Situación 3: 48 employees
    - Remote_Work diferentes
   
    ➡️ Solución: eliminar el primer registro  

- Situación 4: 4 employees
    - Distance_From_Home diferentes
    - Remote_Work diferentes

    ➡️ Solución: eliminar el primer registro

- Situación 5: 8 employees
    - Hourly_Rate -> Not Available
   
    ➡️ Solución: eliminar Hourly_Rate == Not Available     

- Situación 6: 12 employees
    - Distance_From_Home diferentes
    - Distance_From_Home = Environment_Satisfaction
   
    ➡️ Solución: eliminar Distance_From_Home = Environment_Satisfaction


In [190]:
# Importamos librerias
import pandas as pd
import numpy as np

# Ajustes de visualización
pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',None)

In [191]:
# Leemos el .csv y lo guardamos en una variable df
df_original = pd.read_csv('HR RAW DATA.csv', index_col=0)

In [192]:
#creamos una copia del df con la que trabajaremos
df=df_original.copy()

In [193]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1614 entries, 0 to 1613
Data columns (total 41 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       1614 non-null   object 
 1   Attrition                 1614 non-null   object 
 2   BusinessTravel            842 non-null    object 
 3   DailyRate                 1614 non-null   object 
 4   Department                302 non-null    object 
 5   DistanceFromHome          1614 non-null   int64  
 6   Education                 1614 non-null   int64  
 7   EducationField            869 non-null    object 
 8   employeecount             1614 non-null   int64  
 9   employeenumber            1183 non-null   object 
 10  EnvironmentSatisfaction   1614 non-null   int64  
 11  Gender                    1614 non-null   int64  
 12  HourlyRate                1614 non-null   object 
 13  JobInvolvement            1614 non-null   int64  
 14  JobLevel

## Paso 1: Limpieza de datos

### 1.1. Modificar nombre columnas


In [194]:
# def cambio_col_names(col_name):

    

#     return col_name

In [195]:
# cambiamos el nombre a las columnas 
dict_nombre_columnas= {'BusinessTravel':'Business_Travel',
                       'DailyRate': 'Daily_Rate',
                       'DistanceFromHome': 'Distance_From_Home',
                       'EducationField': 'Education_Field',
                       'employeenumber':'Employee_Number', 
                       'EnvironmentSatisfaction': 'Environment_Satisfaction', 
                       'HourlyRate': 'Hourly_Rate', 
                       'JobInvolvement': 'Job_Involvement', 
                       'JobLevel': 'Job_Level', 
                       'JobRole': 'Job_Role', 
                       'JobSatisfaction': 'Job_Satisfaction', 
                       'MaritalStatus': 'Marital_Status',
                       'MonthlyIncome': 'Monthly_Income',
                       'MonthlyRate': 'Monthly_Rate',
                       'NUMCOMPANIESWORKED':'Num_Companies_Worked',
                       'Over18':'Over_18',
                       'OverTime':'Over_Time',
                       'PercentSalaryHike':'Percent_Salary_Hike',
                       'PerformanceRating':'Performance_Rating',
                       'RelationshipSatisfaction':'Relationship_Satisfaction',
                       'StandardHours':'Standard_Hours',
                       'StockOptionLevel':'Stock_Option_Level',
                       'TOTALWORKINGYEARS':'Total_Working_Years',
                       'TrainingTimesLastYear':'Training_Times_Last_Year',
                       'WORKLIFEBALANCE':'Work_Life_Balance',
                       'YearsAtCompany': 'Years_At_Company', 
                       'YearsSinceLastPromotion': 'Years_Since_Last_Promotion', 
                       'YEARSWITHCURRMANAGER': 'Years_With_Curr_Manager', 
                       'DateBirth': 'Date_Birth', 
                       'RemoteWork': 'Remote_Work'
                       }
df.rename(columns=dict_nombre_columnas, inplace=True)

### 1.2. Eliminar columnas no necesarias

In [196]:
lista_col_eliminar=['Department','employeecount','SameAsMonthlyIncome','Salary', 'RoleDepartament', 'NUMBERCHILDREN','YearsInCurrentRole']

for col in lista_col_eliminar:
    df.drop(col, axis=1, inplace=True)

In [197]:
df.shape

(1614, 34)

### 1.3. Crear columnas nuevas

In [198]:
df.insert(1, 'Age_Group', '')

In [199]:
df.shape

(1614, 35)

### 1.4. Modificar valores en columnas

1.4.1 Convertir a str

In [200]:
lista_col_to_str=['Daily_Rate','Employee_Number','Distance_From_Home','Environment_Satisfaction','Gender','Performance_Rating','Remote_Work','Standard_Hours','Total_Working_Years','Work_Life_Balance' ]

for col in lista_col_to_str:
    df[col] = df[col].astype(str)

1.4.2 Cambio de ',' por '.'

In [201]:
lista_col_to_punto=['Daily_Rate','Monthly_Income','Standard_Hours']

for col in lista_col_to_punto:
    df[col] = df[col].str.replace(',','.')

1.4.3 Cambio de ',0' por ''

In [202]:
lista_col_to_no_coma=['Employee_Number', 'Performance_Rating','Total_Working_Years','Work_Life_Balance']

for col in lista_col_to_no_coma:
    df[col] = df[col].str.replace(',0','')

1.4.4 Booleanos

In [203]:
dict_booleanos = {'1': True, 
                  'Yes': True, 
                  'True':True, 
                  '0': False, 
                  'No':False,
                  'False':False}

lista_col_to_booleanos = ['Attrition', 'Over_Time', 'Remote_Work']

for col in lista_col_to_booleanos:
    df[col] = df[col].replace(dict_booleanos)

1.4.5 Otros cambios

In [204]:
reemplazos = {
    'forty-seven': 47,
    'fifty-eight': 58,
    'thirty-six': 36,
    'fifty-five': 55,
    'fifty-two': 52,
    'thirty-one': 31,
    'thirty': 30,
    'twenty-six': 26,
    'thirty-seven': 37,
    'thirty-two': 32,
    'twenty-four': 24
}

df['Age'] = df['Age'].replace(reemplazos)
df['Business_Travel'] = df['Business_Travel'].str.replace('-', '_').str.title()
df['Daily_Rate'] = df['Daily_Rate'].str.replace('$', '')
df['Distance_From_Home'] = df['Distance_From_Home'].str.replace('-', '')
df['Gender'] = df['Gender'].replace({'0': 'M', '1': 'F'})
df['Job_Role'] = df['Job_Role'].str.title()
df['Marital_Status']= df['Marital_Status'].replace({'Marreid':'Married'}).str.title()


/var/folders/g3/22l1jqy501nblpbhtydz9b0w0000gn/T/ipykernel_4569/1484104590.py:17: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Daily_Rate'] = df['Daily_Rate'].str.replace('$', '')


1.4.6 Cambio de nulos


In [205]:
lista_col_int_nulos=['Employee_Number','Hourly_Rate','Performance_Rating','Total_Working_Years','Work_Life_Balance']

for col in lista_col_int_nulos:
    df[col]=df[col].replace({'nan':99999,'NaN':99999, 'Not Available':99999})


1.4.7 Cambio a int

In [206]:
lista_col_to_int=['Age','Distance_From_Home','Employee_Number','Environment_Satisfaction','Hourly_Rate','Job_Involvement','Total_Working_Years','Performance_Rating','Work_Life_Balance']

for col in lista_col_to_int:
    df[col] = df[col].astype(int)

1.4.8 Cambio a float

In [207]:
lista_col_to_float=['Daily_Rate','Monthly_Income','Monthly_Rate','Standard_Hours']

for col in lista_col_to_float:
    df[col] = df[col].astype(float)

1.4.9 Cambio a Bool

In [208]:
lista_col_to_bool=['Remote_Work']

for col in lista_col_to_bool:
    df[col] = df[col].astype(bool)

## Paso 2:Eliminamos duplicados

In [209]:
#Obtenemos el listado del nombre de las columnas del DF
lista_col_names = set(df.columns.tolist())
len(lista_col_names)

35

In [210]:
#Obtenemos un df de los duplicados
df_duplicados = df[df['Employee_Number'].isin(df['Employee_Number'][df['Employee_Number'].duplicated()])].sort_values('Employee_Number')
df_duplicados

,Age,Age_Group,Attrition,Business_Travel,Daily_Rate,Distance_From_Home,Education,Education_Field,Employee_Number,Environment_Satisfaction,Gender,Hourly_Rate,Job_Involvement,Job_Level,Job_Role,Job_Satisfaction,Marital_Status,Monthly_Income,Monthly_Rate,Num_Companies_Worked,Over_18,Over_Time,Percent_Salary_Hike,Performance_Rating,Relationship_Satisfaction,Standard_Hours,Stock_Option_Level,Total_Working_Years,Training_Times_Last_Year,Work_Life_Balance,Years_At_Company,Years_Since_Last_Promotion,Years_With_Curr_Manager,Date_Birth,Remote_Work
82,38,,False,NaN,119.0,3,3,Life Sciences,307,1,M,76,3,3,Sales Executive,3,Divorced,NaN,9647.0,0,Y,NaN,12,3,3,NaN,2,99999,6,2,16,5,13,1985,True
1474,38,,False,NaN,119.0,3,3,Life Sciences,307,1,M,76,3,3,Sales Executive,3,Divorced,NaN,9647.0,0,Y,NaN,12,3,3,NaN,2,99999,6,2,16,5,13,1985,True
1476,55,,False,Travel_Rarely,452.0,1,3,Medical,374,4,M,81,3,5,Manager,1,Single,19045.0,18938.0,0,Y,NaN,14,3,3,NaN,0,37,2,3,36,4,13,1968,True
84,55,,False,Travel_Rarely,452.0,1,3,Medical,374,4,M,81,3,5,Manager,1,Single,19045.0,18938.0,0,Y,NaN,14,3,3,NaN,0,37,2,3,36,4,13,1968,True
1542,47,,True,Non_Travel,666.0,29,4,Life Sciences,376,1,M,88,3,3,Manager,2,Married,NaN,10268.0,1,NaN,NaN,12,3,4,NaN,1,99999,2,2,10,9,9,1976,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614,35,,False,Travel_Rarely,528.0,8,4,Technical Degree,99999,3,M,100,3,1,Human Resources,3,NaN,4323.0,7108.0,1,Y,False,17,3,2,NaN,0,6,2,1,5,1,4,1988,True
610,54,,False,NaN,1082.0,2,4,Life Sciences,99999,3,F,41,2,3,Sales Executive,3,Married,NaN,8392.0,6,Y,False,11,3,2,80.0,1,13,4,3,9,7,0,1969,True
607,30,,False,Travel_Rarely,231.0,8,2,NaN,99999,3,M,62,3,3,Sales Executive,3,NaN,NaN,9977.0,5,NaN,False,11,3,1,NaN,1,99999,2,4,8,7,7,1993,False
661,31,,False,Non_Travel,NaN,5,3,Medical,99999,2,M,74,3,2,Manufacturing Director,1,NaN,9936.0,3787.0,0,Y,False,19,3,2,NaN,0,10,2,3,9,1,7,1992,True


In [211]:
#Comprobamos cuantos duplicados tenemos en 'Employee_Number' 
df['Employee_Number'].duplicated().sum()

534

In [212]:
#Comprobamos la cantidad que tenemos de 'Employee_Number'==99999
df[df['Employee_Number']==99999].shape[0]

431

### 2.1 Situación 1: Todas las columnas son iguales

In [213]:
#Comprobamos el número de duplicados en el DF
df.duplicated().sum()

51

In [214]:
#Eliminamos los duplicados 
df.drop_duplicates(keep='last',inplace=True)

In [215]:
#Comprobamos el número de duplicados en el DF
df.duplicated().sum()

0

### 2.2 Situación 2:Columna 'Distance_From_Home' diferente

In [216]:
#Creamos la lista con el subset de columnas a analizar
lista_columnas_2 = set(lista_col_names)
lista_columnas_2.remove('Distance_From_Home')
len(lista_columnas_2)

34

In [217]:
#Comprobamos el numero de duplicados en el subset
df[lista_columnas_2].duplicated().sum()

13

In [218]:
#Eliminamos los duplicados del subset
df.drop_duplicates(subset=lista_columnas_2, keep='last',inplace=True)

In [219]:
#Comprobamos el numero de duplicados en el subset
df[lista_columnas_2].duplicated().sum()

0

### 2.3 Situación 3: Columnas 'Remote_Work' diferente

In [220]:
#Creamos la lista con el subset de columnas a analizar
lista_columnas_3 = set(lista_col_names)
lista_columnas_3.remove('Remote_Work')
len(lista_columnas_3)

34

In [221]:
#Comprobamos el numero de duplicados en el subset
df[lista_columnas_3].duplicated().sum()

48

In [222]:
#Eliminamos los duplicados del subset
df.drop_duplicates(subset=lista_columnas_3,keep='last',inplace=True)

In [223]:
#Comprobamos el numero de duplicados en el subset
df[lista_columnas_3].duplicated().sum()

0

### 2.4 Situación 4: Columna 'Distance_From_Home' & 'Remote_Work' diferentes

In [224]:
#Creamos la lista con el subset de columnas a analizar
lista_columnas_4 = set(lista_col_names)
lista_columnas_4.remove('Distance_From_Home')
lista_columnas_4.remove('Remote_Work')
len(lista_columnas_4)

33

In [225]:
#Comprobamos el numero de duplicados en el subset
df[lista_columnas_4].duplicated().sum()

6

In [226]:
#Eliminamos los duplicados del subset
df.drop_duplicates(subset=lista_columnas_4,keep='last',inplace=True)

In [187]:
#Comprobamos el numero de duplicados en el subset
df[lista_columnas_4].duplicated().sum()

0

### 2.5 Situación 5: Columna 'Hourly_Rate' == 'Not_Available'

In [227]:
# Creamos un DF de duplicados nuevo
df_duplicados_s5 = df[df['Employee_Number'].isin(df['Employee_Number'][df['Employee_Number'].duplicated() ])].sort_values('Employee_Number')
df_duplicados_s5

,Age,Age_Group,Attrition,Business_Travel,Daily_Rate,Distance_From_Home,Education,Education_Field,Employee_Number,Environment_Satisfaction,Gender,Hourly_Rate,Job_Involvement,Job_Level,Job_Role,Job_Satisfaction,Marital_Status,Monthly_Income,Monthly_Rate,Num_Companies_Worked,Over_18,Over_Time,Percent_Salary_Hike,Performance_Rating,Relationship_Satisfaction,Standard_Hours,Stock_Option_Level,Total_Working_Years,Training_Times_Last_Year,Work_Life_Balance,Years_At_Company,Years_Since_Last_Promotion,Years_With_Curr_Manager,Date_Birth,Remote_Work
302,55,,False,Travel_Rarely,147.0,20,2,Technical Degree,389,2,M,37,3,2,Laboratory Technician,4,Married,NaN,15972.0,3,Y,NaN,19,99999,4,NaN,1,12,4,3,10,0,8,1968,True
1545,55,,False,Travel_Rarely,147.0,17,2,Technical Degree,389,17,M,37,3,2,Laboratory Technician,4,Married,NaN,15972.0,3,Y,NaN,19,99999,4,NaN,1,12,4,3,10,0,8,1968,True
1512,31,,False,Travel_Rarely,691.0,5,4,NaN,423,3,M,86,3,1,Research Scientist,4,Married,NaN,10077.0,0,NaN,NaN,12,3,3,NaN,1,99999,4,3,5,0,3,1992,True
871,31,,False,Travel_Rarely,691.0,11,4,NaN,423,11,M,86,3,1,Research Scientist,4,Married,NaN,10077.0,0,NaN,NaN,12,3,3,NaN,1,99999,4,3,5,0,3,1992,True
874,52,,True,NaN,723.0,17,4,Medical,433,17,M,85,2,2,Research Scientist,2,NaN,4941.0,17747.0,2,NaN,False,15,99999,1,80.0,0,11,3,2,8,7,7,1971,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579,50,,False,NaN,797.0,4,1,NaN,99999,1,M,96,3,5,Research Director,2,Divorced,NaN,15815.0,3,NaN,False,14,3,1,NaN,2,28,4,2,10,1,6,1973,False
562,35,,False,Travel_Rarely,1214.0,1,3,Medical,99999,2,M,30,2,1,Research Scientist,3,Single,NaN,26278.0,1,NaN,False,18,3,1,NaN,0,6,3,3,6,0,4,1988,False
561,32,,False,Travel_Rarely,548.0,1,3,Life Sciences,99999,2,M,66,3,2,Research Scientist,2,NaN,NaN,7346.0,1,Y,False,17,3,2,80.0,2,10,3,3,10,0,9,1991,True
659,41,,False,NaN,930.0,3,3,Life Sciences,99999,3,M,57,2,2,Sales Executive,2,Divorced,8938.0,12227.0,2,Y,NaN,11,3,3,NaN,1,14,5,3,5,0,4,1982,False


In [229]:
# Filrtamos el DF para obtener el DF dónde el 'Hourly_Rate'==99999 y el 'Employee_Number' != 999999
df_duplicados_s5_filtrado = df_duplicados_s5[df_duplicados_s5['Hourly_Rate']==99999]
df_duplicados_s5_filtrado = df_duplicados_s5_filtrado.loc[df_duplicados_s5_filtrado['Employee_Number']!=99999,:]
df_duplicados_s5_filtrado


,Age,Age_Group,Attrition,Business_Travel,Daily_Rate,Distance_From_Home,Education,Education_Field,Employee_Number,Environment_Satisfaction,Gender,Hourly_Rate,Job_Involvement,Job_Level,Job_Role,Job_Satisfaction,Marital_Status,Monthly_Income,Monthly_Rate,Num_Companies_Worked,Over_18,Over_Time,Percent_Salary_Hike,Performance_Rating,Relationship_Satisfaction,Standard_Hours,Stock_Option_Level,Total_Working_Years,Training_Times_Last_Year,Work_Life_Balance,Years_At_Company,Years_Since_Last_Promotion,Years_With_Curr_Manager,Date_Birth,Remote_Work
892,42,,False,NaN,201.0,1,4,NaN,517,2,F,99999,3,1,Laboratory Technician,1,NaN,NaN,20490.0,3,Y,False,16,3,2,80.0,1,8,5,3,5,1,2,1981,True
1566,35,,False,Travel_Rarely,1276.0,16,3,Life Sciences,586,4,M,99999,3,3,Healthcare Representative,3,NaN,NaN,14295.0,4,NaN,True,12,3,3,NaN,0,10,2,3,8,0,0,1988,True
1578,29,,False,NaN,806.0,1,4,Life Sciences,710,2,M,99999,1,1,Research Scientist,4,Divorced,NaN,18959.0,1,Y,NaN,18,3,4,NaN,1,10,5,3,10,2,8,1994,True
340,27,,False,Travel_Rarely,NaN,1,2,Life Sciences,725,3,F,99999,3,3,Manufacturing Director,1,Married,NaN,6009.0,1,NaN,NaN,11,3,4,NaN,0,9,3,3,9,0,7,1996,False
1604,41,,True,NaN,1085.0,2,4,Life Sciences,927,2,F,99999,1,1,Laboratory Technician,4,Divorced,NaN,17725.0,4,NaN,True,13,3,3,80.0,1,99999,1,2,7,1,0,1982,True
1605,31,,False,Travel_Rarely,154.0,7,4,NaN,941,2,M,99999,2,1,Sales Representative,3,NaN,2329.0,11737.0,3,NaN,NaN,15,99999,2,80.0,0,99999,2,4,7,5,2,1992,True
987,33,,False,Non_Travel,1038.0,8,1,Life Sciences,1044,2,F,99999,2,1,Sales Representative,4,NaN,NaN,21437.0,0,Y,NaN,19,3,4,NaN,0,99999,2,2,2,2,2,1990,False
1008,35,,False,Travel_Frequently,NaN,4,4,Other,1185,4,M,99999,2,1,Laboratory Technician,4,Married,2376.0,26537.0,1,NaN,NaN,13,3,2,NaN,1,2,2,4,2,2,2,1988,True


In [232]:
#Guardamos en una lista los números de los índices de todas las filas del DF filtrado 'df_duplicados_s5_filtrado'
lista_index_situacion_5= df_duplicados_s5_filtrado.index.tolist()

In [233]:
#Utilizamos un for loop para iterar por la lista de los números de índice y eliminamos cada uno de estos de nuestro DF original
for employee in lista_index_situacion_5:
    df.drop(employee,axis=0,inplace=True)

In [234]:
df['Employee_Number'].duplicated().sum()

408

### 2.6 Situación 6: Columna 'Distance_From_Home' == 'Environment_Satisfaction'

In [235]:
# Creamos un DF de duplicados nuevo
df_duplicados_s6 = df[df['Employee_Number'].isin(df['Employee_Number'][df['Employee_Number'].duplicated() ])].sort_values('Employee_Number')
df_duplicados_s6

,Age,Age_Group,Attrition,Business_Travel,Daily_Rate,Distance_From_Home,Education,Education_Field,Employee_Number,Environment_Satisfaction,Gender,Hourly_Rate,Job_Involvement,Job_Level,Job_Role,Job_Satisfaction,Marital_Status,Monthly_Income,Monthly_Rate,Num_Companies_Worked,Over_18,Over_Time,Percent_Salary_Hike,Performance_Rating,Relationship_Satisfaction,Standard_Hours,Stock_Option_Level,Total_Working_Years,Training_Times_Last_Year,Work_Life_Balance,Years_At_Company,Years_Since_Last_Promotion,Years_With_Curr_Manager,Date_Birth,Remote_Work
302,55,,False,Travel_Rarely,147.0,20,2,Technical Degree,389,2,M,37,3,2,Laboratory Technician,4,Married,NaN,15972.0,3,Y,NaN,19,99999,4,NaN,1,12,4,3,10,0,8,1968,True
1545,55,,False,Travel_Rarely,147.0,17,2,Technical Degree,389,17,M,37,3,2,Laboratory Technician,4,Married,NaN,15972.0,3,Y,NaN,19,99999,4,NaN,1,12,4,3,10,0,8,1968,True
1512,31,,False,Travel_Rarely,691.0,5,4,NaN,423,3,M,86,3,1,Research Scientist,4,Married,NaN,10077.0,0,NaN,NaN,12,3,3,NaN,1,99999,4,3,5,0,3,1992,True
871,31,,False,Travel_Rarely,691.0,11,4,NaN,423,11,M,86,3,1,Research Scientist,4,Married,NaN,10077.0,0,NaN,NaN,12,3,3,NaN,1,99999,4,3,5,0,3,1992,True
1515,52,,True,NaN,723.0,8,4,Medical,433,3,M,85,2,2,Research Scientist,2,NaN,4941.0,17747.0,2,NaN,False,15,99999,1,80.0,0,11,3,2,8,7,7,1971,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561,32,,False,Travel_Rarely,548.0,1,3,Life Sciences,99999,2,M,66,3,2,Research Scientist,2,NaN,NaN,7346.0,1,Y,False,17,3,2,80.0,2,10,3,3,10,0,9,1991,True
559,35,,False,Travel_Rarely,809.0,16,3,Medical,99999,1,M,84,4,1,Laboratory Technician,2,Married,NaN,16479.0,0,NaN,NaN,13,3,3,NaN,1,6,5,3,5,0,3,1988,False
558,41,,True,NaN,1102.0,1,2,NaN,99999,2,F,94,3,2,Sales Executive,4,NaN,NaN,19479.0,8,NaN,True,11,3,1,NaN,0,8,0,1,6,0,5,1982,False
595,31,,False,Travel_Rarely,688.0,7,3,NaN,99999,3,M,44,2,3,Manager,4,Divorced,NaN,25291.0,9,Y,NaN,21,4,3,80.0,1,10,3,2,5,0,1,1992,True


In [237]:
# Filtramos el DF para obtener el DF dónde el 'Distance_From_Home' == 'Environment_Satisfaction'
# y el 'Employee_Number' != 999999
df_duplicados_s6_filtrado = df_duplicados_s6[(df_duplicados_s6['Distance_From_Home']==df_duplicados_s6['Environment_Satisfaction'])]
df_duplicados_s6_filtrado = df_duplicados_s6_filtrado.loc[df_duplicados_s6_filtrado['Employee_Number']!=99999,:]
df_duplicados_s6_filtrado

,Age,Age_Group,Attrition,Business_Travel,Daily_Rate,Distance_From_Home,Education,Education_Field,Employee_Number,Environment_Satisfaction,Gender,Hourly_Rate,Job_Involvement,Job_Level,Job_Role,Job_Satisfaction,Marital_Status,Monthly_Income,Monthly_Rate,Num_Companies_Worked,Over_18,Over_Time,Percent_Salary_Hike,Performance_Rating,Relationship_Satisfaction,Standard_Hours,Stock_Option_Level,Total_Working_Years,Training_Times_Last_Year,Work_Life_Balance,Years_At_Company,Years_Since_Last_Promotion,Years_With_Curr_Manager,Date_Birth,Remote_Work
1545,55,,False,Travel_Rarely,147.0,17,2,Technical Degree,389,17,M,37,3,2,Laboratory Technician,4,Married,NaN,15972.0,3,Y,NaN,19,99999,4,NaN,1,12,4,3,10,0,8,1968,True
871,31,,False,Travel_Rarely,691.0,11,4,NaN,423,11,M,86,3,1,Research Scientist,4,Married,NaN,10077.0,0,NaN,NaN,12,3,3,NaN,1,99999,4,3,5,0,3,1992,True
874,52,,True,NaN,723.0,17,4,Medical,433,17,M,85,2,2,Research Scientist,2,NaN,4941.0,17747.0,2,NaN,False,15,99999,1,80.0,0,11,3,2,8,7,7,1971,False
886,29,,False,NaN,1283.0,17,3,NaN,495,17,M,54,3,1,Research Scientist,4,NaN,NaN,18168.0,9,NaN,False,16,99999,4,NaN,0,6,4,3,3,1,2,1994,False
1560,51,,False,Travel_Rarely,1178.0,31,2,NaN,500,31,F,87,3,2,Sales Executive,4,NaN,NaN,14862.0,4,NaN,False,11,3,3,NaN,1,18,2,2,7,0,7,1972,False
90,50,,False,Travel_Rarely,1099.0,27,4,Life Sciences,569,27,M,88,2,4,Manager,3,Married,17046.0,9314.0,0,Y,False,15,3,2,NaN,1,28,2,3,27,15,7,1973,True
1590,30,,False,NaN,501.0,35,5,Marketing,747,35,M,99,3,2,Sales Executive,4,Divorced,5304.0,25275.0,7,Y,False,23,99999,4,80.0,1,10,2,2,8,7,7,1993,False
363,26,,False,Travel_Frequently,1283.0,28,3,Medical,956,28,M,52,2,2,Sales Executive,1,NaN,4294.0,11148.0,1,Y,NaN,12,3,2,NaN,0,99999,2,3,7,0,7,1997,True
367,45,,False,Non_Travel,805.0,47,2,NaN,972,47,M,57,3,2,Laboratory Technician,2,NaN,4447.0,23163.0,1,NaN,NaN,12,3,2,NaN,0,99999,5,2,9,0,8,1978,True
990,46,,False,NaN,430.0,47,4,Medical,1069,47,M,40,3,5,Research Director,4,NaN,NaN,21445.0,9,NaN,False,17,3,4,80.0,2,23,0,3,2,2,2,1977,True


In [238]:
# Guardamos en una lista los números de los índices de todas las filas del DF filtrado 'df_duplicados_s6_filtrado'
lista_index_situacion_6=df_duplicados_s6_filtrado.index.tolist()

In [239]:
#Utilizamos un for loop para iterar por la lista de los números de índice y eliminamos cada uno de estos de nuestro DF original
for employee in lista_index_situacion_6:
    df.drop(employee,axis=0,inplace=True)

## Paso 3: Cambio 'Employee_Number' Nulos

In [240]:
# Mostramos del DF original los 'Employee_number' == 99999
df[df['Employee_Number']==99999]

,Age,Age_Group,Attrition,Business_Travel,Daily_Rate,Distance_From_Home,Education,Education_Field,Employee_Number,Environment_Satisfaction,Gender,Hourly_Rate,Job_Involvement,Job_Level,Job_Role,Job_Satisfaction,Marital_Status,Monthly_Income,Monthly_Rate,Num_Companies_Worked,Over_18,Over_Time,Percent_Salary_Hike,Performance_Rating,Relationship_Satisfaction,Standard_Hours,Stock_Option_Level,Total_Working_Years,Training_Times_Last_Year,Work_Life_Balance,Years_At_Company,Years_Since_Last_Promotion,Years_With_Curr_Manager,Date_Birth,Remote_Work
3,47,,False,Travel_Rarely,359.0,2,4,Medical,99999,1,F,82,3,4,Research Director,3,Married,17169.0,26703.0,3,Y,NaN,19,3,2,NaN,2,99999,2,99999,20,5,6,1976,False
4,46,,False,NaN,1319.0,3,3,Technical Degree,99999,1,F,45,4,4,Sales Executive,1,Divorced,NaN,7739.0,2,Y,False,12,3,4,NaN,1,99999,5,3,19,2,8,1977,False
8,41,,False,NaN,1276.0,2,5,NaN,99999,2,F,91,3,4,Manager,1,Married,16595.0,5626.0,7,NaN,False,16,3,2,NaN,1,22,2,3,18,11,8,1982,True
11,56,,False,Travel_Rarely,1369.0,23,3,Life Sciences,99999,4,M,68,3,4,Manufacturing Director,2,Married,13402.0,18235.0,4,NaN,True,12,99999,1,NaN,1,33,0,3,19,15,9,1967,False
12,38,,False,NaN,201.0,10,3,Medical,99999,2,F,99,1,3,Research Director,3,Married,13206.0,3376.0,3,Y,False,12,3,1,NaN,1,99999,3,3,18,1,11,1985,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1601,28,,False,NaN,760.0,43,4,NaN,99999,43,F,81,3,2,Sales Executive,2,NaN,NaN,3698.0,1,Y,NaN,20,4,1,NaN,0,8,2,3,8,7,5,1995,False
1602,26,,True,NaN,NaN,5,2,Medical,99999,3,F,88,2,1,Research Scientist,3,Married,2366.0,20898.0,1,NaN,NaN,14,3,1,NaN,1,8,2,3,8,1,7,1997,True
1607,31,,False,NaN,616.0,12,3,Medical,99999,4,F,41,3,2,Healthcare Representative,4,NaN,NaN,17369.0,0,NaN,NaN,11,3,3,NaN,2,10,2,1,9,8,5,1992,True
1611,39,,False,Travel_Rarely,903.0,13,5,NaN,99999,13,M,41,4,3,Sales Executive,3,Single,NaN,2560.0,0,NaN,False,18,3,4,NaN,0,9,3,3,8,0,7,1984,True


In [244]:
# Obtenemos una lista de los todos los 'Employee_Number' y la ordenamos de manera ascendente
lista_employee_number = df['Employee_Number'].unique().tolist()
lista_employee_number.sort()

In [245]:
# Obtenemos el número del último 'Employee_Number' sin tener en cuenta el 99999
lista_employee_number.remove(99999)
last_employee_number = lista_employee_number[-1]
last_employee_number

2068

In [246]:
# Creamos la variable del primer nuevo número de 'Employee_Number' y
# del número de los índices de los 'Employee_Number' == 99999
new_employee_number = last_employee_number + 1
indices = df.index[df['Employee_Number'] == 99999]

#Iteramos con un for loop por la lista de índices y cambiamos el valor del 'Employee_Number' por el nuevo valor
for i in indices:
    df.loc[i, 'Employee_Number'] = new_employee_number
    new_employee_number += 1

In [247]:
#Comprobamos el tamaño final del DF
df.shape

(1477, 35)

In [249]:
#Comprobamos si quedan duplicados en la columna de 'Employee_Number'
df['Employee_Number'].duplicated().sum()

0

In [250]:
#Adjudicamos la columna 'Employee_Number' cómo la columna de ínidce y eliminamos el actual índice
df.set_index('Employee_Number',drop=True,inplace=True)

In [254]:
#Visualizamos el DF resultante
df.head(5)

,Age,Age_Group,Attrition,Business_Travel,Daily_Rate,Distance_From_Home,Education,Education_Field,Environment_Satisfaction,Gender,Hourly_Rate,Job_Involvement,Job_Level,Job_Role,Job_Satisfaction,Marital_Status,Monthly_Income,Monthly_Rate,Num_Companies_Worked,Over_18,Over_Time,Percent_Salary_Hike,Performance_Rating,Relationship_Satisfaction,Standard_Hours,Stock_Option_Level,Total_Working_Years,Training_Times_Last_Year,Work_Life_Balance,Years_At_Company,Years_Since_Last_Promotion,Years_With_Curr_Manager,Date_Birth,Remote_Work
Employee_Number,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
162,51,,False,NaN,684.0,6,3,NaN,1,M,51,3,5,Research Director,3,NaN,19537.0,6462.0,7,Y,False,13,3,3,NaN,0,99999,5,3,20,15,15,1972,True
259,52,,False,NaN,699.0,1,4,Life Sciences,3,M,65,2,5,Manager,3,NaN,19999.0,5678.0,0,NaN,NaN,14,3,1,NaN,1,34,5,3,33,11,9,1971,True
319,42,,False,Travel_Rarely,532.0,4,2,Technical Degree,3,M,58,3,5,Manager,4,Married,19232.0,4933.0,1,NaN,False,11,3,4,NaN,0,22,3,99999,22,11,15,1981,True
2069,47,,False,Travel_Rarely,359.0,2,4,Medical,1,F,82,3,4,Research Director,3,Married,17169.0,26703.0,3,Y,NaN,19,3,2,NaN,2,99999,2,99999,20,5,6,1976,False
2070,46,,False,NaN,1319.0,3,3,Technical Degree,1,F,45,4,4,Sales Executive,1,Divorced,NaN,7739.0,2,Y,False,12,3,4,NaN,1,99999,5,3,19,2,8,1977,False


In [255]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1477 entries, 162 to 2466
Data columns (total 34 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Age                         1477 non-null   int64  
 1   Age_Group                   1477 non-null   object 
 2   Attrition                   1477 non-null   bool   
 3   Business_Travel             768 non-null    object 
 4   Daily_Rate                  1365 non-null   float64
 5   Distance_From_Home          1477 non-null   int64  
 6   Education                   1477 non-null   int64  
 7   Education_Field             789 non-null    object 
 8   Environment_Satisfaction    1477 non-null   int64  
 9   Gender                      1477 non-null   object 
 10  Hourly_Rate                 1477 non-null   int64  
 11  Job_Involvement             1477 non-null   int64  
 12  Job_Level                   1477 non-null   int64  
 13  Job_Role                    147

In [258]:
df.describe(include=['O','int','float']).T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Age,1477.0,NaN,NaN,NaN,36.927556,9.127865,18.0,30.0,36.0,43.0,60.0
Age_Group,1477,1,,1477,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Business_Travel,768,3,Travel_Rarely,536,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Daily_Rate,1365.0,NaN,NaN,NaN,805.838095,401.291543,103.0,472.0,805.0,1157.0,1499.0
Distance_From_Home,1477.0,NaN,NaN,NaN,11.205823,10.434121,1.0,2.0,8.0,17.0,49.0
Education,1477.0,NaN,NaN,NaN,2.914015,1.024788,1.0,2.0,3.0,4.0,5.0
Education_Field,789,6,Life Sciences,322,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Environment_Satisfaction,1477.0,NaN,NaN,NaN,4.245768,6.870353,1.0,2.0,3.0,4.0,49.0
Gender,1477,2,M,885,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Hourly_Rate,1477.0,NaN,NaN,NaN,5208.141503,22085.204556,30.0,49.0,68.0,86.0,99999.0


In [261]:
#Guardamos el DF en formato .csv
df.to_csv('HR_CLEAN_DATA.csv', index=True)